# 🐳 DataTalks DE Zoomcamp - Module 1 : Introduction to Docker

> **Objectif du module** : Comprendre Docker et containerisation

> **Durée** : Semaine 1 

> **Date de mes notes** : Janvier 2026


## 📋 Table des Matières
[TOC auto-générée]

# 🐳 Docker : Qu'est-ce que c'est ?

## ✅ Ce QUE Docker EST

### 🎯 Définition
**Docker est une plateforme de containerisation** qui permet d'empaqueter une application avec toutes ses dépendances dans un container isolé et portable.

### 🔑 Caractéristiques principales

| Aspect | Description |
|--------|-------------|
| **🏗️ Container Runtime** | Exécute des applications isolées sur un seul kernel |
| **📦 Packaging Tool** | Emballe app + dépendances + config dans une image |
| **⚡ Léger** | Partage le kernel de l'OS hôte (pas d'OS complet) |
| **🚀 Rapide** | Démarre en millisecondes (c'est juste un processus) |
| **🔄 Portable** | "Works on my machine" → "Works everywhere" |
| **📏 Isolation** | Via namespaces et cgroups du kernel Linux |

### 💡 En bref
> Docker permet de créer des **environnements reproductibles et isolés** avec un overhead minimal (~2-5%)

---

## ❌ Ce QUE Docker N'EST PAS

### 🚫 Confusions courantes

#### 1. **PAS une Machine Virtuelle**
- ❌ Docker ne virtualise **pas** le hardware
- ❌ Docker n'embarque **pas** d'OS complet avec kernel
- ✅ Docker partage le kernel de l'hôte
- ✅ Beaucoup plus léger qu'une VM (MB vs GB)
```
VM       : App → OS complet → Hypervisor → Hardware
Docker   : App → Docker Engine → OS Hôte → Hardware
```

#### 2. **PAS Kubernetes**
- **Docker** = Containerisation (créer/exécuter des containers sur **1 machine**)
- **Kubernetes** = Orchestration (gérer des milliers de containers sur **des clusters**)
- Relation : K8s peut utiliser Docker, mais ce sont des outils différents

#### 3. **PAS une solution de déploiement complète**
Docker seul ne fournit **PAS** :
- ❌ Load balancing automatique
- ❌ Auto-scaling
- ❌ Service discovery
- ❌ Secrets management avancé
- ❌ Monitoring intégré

**Tu as besoin de :** Docker Compose, Swarm, Kubernetes, ou services cloud managés

#### 4. **PAS un outil de configuration management**
- ❌ Pas un Ansible/Chef/Puppet
- Docker crée des **images immuables** (tu ne modifies jamais un container en prod)
- Config management modifie des **serveurs existants** (mutable infrastructure)

#### 5. **PAS magique pour la sécurité**
- Les containers **partagent le kernel** → vulnérabilité possible
- Root dans un container ≠ isolé de root sur l'hôte
- Nécessite des pratiques de sécurité appropriées

#### 6. **PAS Docker Desktop = Docker Engine**
- **Docker Engine** : Le vrai Docker (open source, Linux natif)
- **Docker Desktop** : App commerciale (Windows/Mac) avec VM Linux incluse

---

## 📊 Comparaison Rapide

| Critère | Docker | VM |
|---------|--------|-----|
| **Taille** | 5-500 MB | 2-20 GB |
| **Démarrage** | < 1 seconde | 30-60 secondes |
| **Overhead** | ~2-5% | ~40-60% |
| **Isolation** | Processus (namespace) | Hardware (hypervisor) |
| **Portabilité** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| **Performance** | Quasi-native | Pénalité significative |

---

## 🎯 Cas d'usage idéaux pour Docker

✅ **Microservices**  
✅ **CI/CD pipelines**  
✅ **Environnements de dev cohérents**  
✅ **Applications web stateless**  
✅ **Tests d'intégration**  

❌ **Applications GUI desktop**  
❌ **Kernel modules / drivers**  
❌ **Systèmes temps-réel**  
❌ **Applications legacy Windows complexes**

---

## 💭 Analogie Simple

**Docker** = Containers de transport maritime
- Standardisés
- Portables (bateau → train → camion)
- Empilables
- Isolés (ce qui est dans un container ne peut pas affecter un autre)
- Légers par rapport au contenu

**VM** = Déménager une maison entière à chaque fois
- Lourde
- Coûteuse
- Complète (tout est dedans)
- Isolation totale

---

## 🔑 Concept clé : Overhead

**Overhead** = Coût supplémentaire (ressources) pour utiliser une technologie

- **Docker** : 2-5% d'overhead → acceptable pour 99% des cas
- **VM** : 40-60% d'overhead → lourd mais isolation maximale

> 💡 L'overhead se mesure en CPU, RAM, disque et temps de démarrage

---

## 📚 Ressources

- [Documentation officielle Docker](https://docs.docker.com/)
- [Docker Hub](https://hub.docker.com/)
- [DataTalks DE Zoomcamp](https://github.com/DataTalksClub/data-engineering-zoomcamp)